In [1]:
import pandas as pd
import datetime
import numpy as np
# функция для раздулки
def get_razd(sx_razd,v_):
    ch_razd = {}
    #     если это деловая часть
    for ind in sx_razd.index:

        if sx_razd.at[ind,'Из какой части пром']==0:
            v___ = v_ * sx_razd.at[ind,'Процент разделки']
    #         df_cons = df_cons.append(pd.DataFrame({'Дата забоя':df['Дата забоя'],'Градация':grad,'Наименование части':fff.at[ind,'Часть'],'Объем':v___}))
            ch_razd[sx_razd.at[ind,'Часть']] =v___


    for ind in sx_razd.index:
    #     для пром
        if sx_razd.at[ind,'Из какой части пром']!=0:
    #         print(fff.at[ind,'Из какой части пром'])
    #         Процент вет брака
            pr_vetbr = sx_razd[(sx_razd['Из какой части пром']==sx_razd.at[ind,'Из какой части пром'])&(sx_razd['Часть']==sx_razd.at[ind,'Часть'])]['Процент разделки'].values
            v_pochasti = ch_razd[sx_razd.at[ind,'Из какой части пром']]
            it_prom = pr_vetbr * v_pochasti
            ch_razd[sx_razd.at[ind,'Часть']] = it_prom
            ch_razd[sx_razd.at[ind,'Из какой части пром']] = ch_razd[sx_razd.at[ind,'Из какой части пром']] - ch_razd[sx_razd.at[ind,'Часть']]
    
#     Выбираем минимальное из деловых частей (там где больше одного элемента)
    for ind in sx_razd.index:
        ch_razd[sx_razd.at[ind,'Часть']] = min(ch_razd[sx_razd.at[ind,'Часть']])
    
    for key___ in ch_razd.keys():
        ch_razd[key___] = ch_razd[key___].sum()
    return ch_razd


# делит по 100 гр
def get_po_sto_gr(inp):
    inp = inp.set_index('Дата забоя')
    df = pd.DataFrame(inp.index)
    df = df.set_index('Дата забоя')
    for ind, col in enumerate(inp.columns):
#         print(col.split('-')[0]=='2,500')
        try:
            if col.split('-')[0]=='2,500':
                df['2,500-3,500'] = inp['2,500-3,500']
#                 print(col)
            
            if ind ==0 or ind%2==0:
                cols = inp.iloc[:,[ind,ind+1]].columns
                df[cols[0].split('-')[0]+'-'+cols[1].split('-')[1]] = inp.iloc[:,[ind,ind+1]].sum(axis = 1)

            
        except IndexError:
            pass
    return df.reset_index()

# логика формирования итогового фрейма

def get_cons_arr(df_gr,dict_,chema_r,dict_sx):

    df_cons = pd.DataFrame()
    for date in df_gr['Дата забоя'].unique():

        df = df_gr[df_gr['Дата забоя']==date]

        for grad in dict_.keys():


            v_razd = df[grad].values * dict_[grad]
            v_cb = df[grad].values * (1- dict_[grad])

    #         Часть с разделкой
            if v_razd>0:

    #             print(v_razd)

    #             Иду по пропорции разделки
                for el_sx in dict_sx.keys():

    #                 Объем по схеме
                    v_ = v_razd * dict_sx[el_sx]
    #                 print(str(v_razd)+' '+ str(v_))
                    sx_razd = chema_r[chema_r['Номер'] == el_sx]
                    it_chema = get_razd(sx_razd,v_)
                    for key_ in it_chema.keys():
                        df_cons = df_cons.append(pd.DataFrame({'Дата забоя':df['Дата забоя'],'Градация':grad,'Наименование части':key_,'Объем':it_chema[key_]}))
    #                     print(key_)
    #                     break
    #                 break



    #                 print('общий объем'+ str(v_razd) + ' Объем на конкретную схему эту '+str(v_))



    #         Часть ЦБ
            if v_cb>0:

                df_cons = df_cons.append(pd.DataFrame({'Дата забоя':df['Дата забоя'],'Градация':grad,'Наименование части':'ЦБ','Объем':v_cb}))
    return df_cons


# формирование чахохбили
def calc_dop_chakh(cons2,from_name,to_name,percent):
    for_ch_okor = cons2.loc[cons2['Наименование части']==from_name]
    ost_okor = cons2.loc[cons2['Наименование части']==from_name]

    for col in for_ch_okor.columns:
        try:
            for_ch_okor.loc[:,col] = for_ch_okor.loc[:,col]*percent
        except TypeError:
            pass

    for col in ost_okor.columns:
        if ost_okor[col].dtype=='float':
            cons2.loc[cons2['Наименование части']==from_name,col] = ost_okor[col]-for_ch_okor[col]

    for_ch_okor['Наименование части'] = to_name

    cons2 = cons2.append(for_ch_okor).reset_index(drop=True)
    return cons2
        

In [2]:
directory = 'C:\\Users\\Кедрун Никита\\PycharmProjects\\analyze_zab\\unput\\'


# какие месяца по дням а какие по месяцам
params = {'days':[8],'month':[9,10,11,12]}

# Сколько с градации брать на разделку


dict_ ={'0,500-0,600':0, 
        '0,600-0,700':0, 
        '0,700-0,800':0, 
        '0,800-0,900':0,
        '0,900-1,000':0, 
        '1,000-1,100':0, 
        '1,100-1,200':0, 
        '1,200-1,300':0.9,
        '1,300-1,400':0.6, 
        '1,400-1,500':0, 
        '1,500-1,600':0.8, 
        '1,600-1,700':0,
        '1,700-1,800':0.5, 
        '1,800-1,900':0, 
        '1,900-2,000':0, 
        '2,000-2,100':0,
        '2,100-2,200':0, 
        '2,200-2,300':0, 
        '2,300-2,400':0, 
        '2,400-2,500':0,
        '2,500-3,500':0}


# пропорция по схемам разделки
dict_sx = {1:0.5,2:0.5}

In [3]:
mpf =  pd.read_excel(directory+'МПФ_заб.xlsx',sheet_name='Забой')
mpf = get_po_sto_gr(mpf)
chema_r = pd.read_excel(directory+'Схемы разделки МПФ.xlsx',sheet_name='Лист1')

In [4]:
# Часть по дням
df_for_days = mpf[[True  if x.month == params['days'][0] else False for x in mpf['Дата забоя']]]
df_for_days_gr = df_for_days.groupby('Дата забоя').sum().reset_index()

In [5]:
df_for_days = get_cons_arr(df_for_days_gr,dict_,chema_r,dict_sx)

In [6]:
# Часть по месяцам
df_for_month = mpf[mpf['Дата забоя'].dt.month.isin(params['month'])]
df_for_month['Дата забоя'] = [datetime.datetime(x.year,x.month,1) for x in df_for_month['Дата забоя']]
df_for_month_gr = df_for_month.groupby('Дата забоя').sum().reset_index()

C:\Users\8777~1\AppData\Local\Temp/ipykernel_15272/3183176384.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_for_month['Дата забоя'] = [datetime.datetime(x.year,x.month,1) for x in df_for_month['Дата забоя']]


In [7]:
df_for_month = get_cons_arr(df_for_month_gr,dict_,chema_r,dict_sx)

In [8]:
cons1 = df_for_days.append(df_for_month)

In [9]:
cons2 = pd.pivot_table(cons1, values = 'Объем',index =['Градация','Наименование части'],columns = 'Дата забоя',aggfunc=np.sum ).reset_index().fillna(0)

In [10]:
cons2 = calc_dop_chakh(cons2,'Окорочек','Сырье для чахохбили',0.7)
cons2 = calc_dop_chakh(cons2,'Кость килевая','Сырье для чахохбили',0.3)

In [11]:
cons2.sort_values(['Наименование части','Градация']).to_excel(directory+' testMPF.xlsx')